# 20. 多任務學習 (MTL) vs 單任務學習 (STL) 比較

## 目的
比較多任務學習（MTL）與單任務學習（STL）在三高疾病預測上的表現。

## 假說
MTL 可能透過學習跨任務的共享表徵來提升預測效能。

## 背景
根據共病分析：
- 三高疾病之間的直接相關性較弱（Phi < 0.1）
- 高血壓 ↔ 高血糖：OR=1.72（中度關聯）
- 高血壓 ↔ 高血脂：OR=1.07（近乎獨立）

## 日期：2026-01-13

In [ ]:
# 安裝 PyTorch（若未安裝）
try:
    import torch
    print(f"PyTorch 已安裝：{torch.__version__}")
except ImportError:
    print("正在安裝 PyTorch...")
    !pip install torch
    print("PyTorch 安裝完成")

In [ ]:
# 匯入套件
import pandas as pd
import numpy as np
import time
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import StratifiedGroupKFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import roc_auc_score
from sklearn.base import clone

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

print("套件載入完成")
print(f"PyTorch 版本：{torch.__version__}")

## 1. 載入資料

In [ ]:
# 載入滑動視窗資料
df = pd.read_csv('../../data/01_primary/SUA/processed/SUA_sliding_window.csv')
print(f"資料：{len(df):,} 筆樣本，{df['patient_id'].nunique():,} 位患者")

# 特徵
feature_cols = [
    'sex', 'Age',
    'FBG_Tinput1', 'TC_Tinput1', 'Cr_Tinput1', 'UA_Tinput1', 'GFR_Tinput1', 'BMI_Tinput1', 'SBP_Tinput1', 'DBP_Tinput1',
    'FBG_Tinput2', 'TC_Tinput2', 'Cr_Tinput2', 'UA_Tinput2', 'GFR_Tinput2', 'BMI_Tinput2', 'SBP_Tinput2', 'DBP_Tinput2',
    'Delta_FBG', 'Delta_TC', 'Delta_Cr', 'Delta_UA', 'Delta_GFR', 'Delta_BMI', 'Delta_SBP', 'Delta_DBP'
]

X = df[feature_cols]
groups = df['patient_id']

# 目標變數（將 1/2 轉換為 0/1）
y_htn = (df['hypertension_target'] == 2).astype(int)
y_hg = (df['hyperglycemia_target'] == 2).astype(int)
y_dl = (df['dyslipidemia_target'] == 2).astype(int)

# MTL 用的組合目標矩陣
Y = pd.DataFrame({
    'HTN': y_htn,
    'HG': y_hg,
    'DL': y_dl
})

print(f"\n特徵數：{len(feature_cols)}")
print(f"任務：{list(Y.columns)}")
print(f"\n盛行率：")
for col in Y.columns:
    print(f"  {col}：{Y[col].mean()*100:.1f}%")

## 2. 定義模型

### STL（單任務學習）
- 3 個獨立模型，各預測一種疾病

### MTL（多任務學習）
- 1 個共享模型，同時預測 3 種疾病

In [ ]:
# PyTorch MTL 模型
class MTLNetwork(nn.Module):
    """多任務學習網路（含共享層）"""
    def __init__(self, input_dim, hidden_dims=[64, 32], n_tasks=3):
        super().__init__()
        
        # 共享層
        layers = []
        prev_dim = input_dim
        for h_dim in hidden_dims:
            layers.extend([
                nn.Linear(prev_dim, h_dim),
                nn.ReLU(),
                nn.Dropout(0.2)
            ])
            prev_dim = h_dim
        self.shared = nn.Sequential(*layers)
        
        # 任務專屬輸出層
        self.heads = nn.ModuleList([
            nn.Linear(hidden_dims[-1], 1) for _ in range(n_tasks)
        ])
    
    def forward(self, x):
        shared_repr = self.shared(x)
        outputs = [torch.sigmoid(head(shared_repr)) for head in self.heads]
        return torch.cat(outputs, dim=1)


class STLNetwork(nn.Module):
    """單任務學習網路（相同架構，但獨立訓練）"""
    def __init__(self, input_dim, hidden_dims=[64, 32]):
        super().__init__()
        
        layers = []
        prev_dim = input_dim
        for h_dim in hidden_dims:
            layers.extend([
                nn.Linear(prev_dim, h_dim),
                nn.ReLU(),
                nn.Dropout(0.2)
            ])
            prev_dim = h_dim
        layers.append(nn.Linear(hidden_dims[-1], 1))
        
        self.network = nn.Sequential(*layers)
    
    def forward(self, x):
        return torch.sigmoid(self.network(x))

print("模型定義完成")

In [ ]:
def train_mtl_model(X_train, Y_train, X_val, Y_val, epochs=100, lr=0.001, batch_size=256):
    """訓練 MTL 模型"""
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # 準備資料
    X_train_t = torch.FloatTensor(X_train).to(device)
    Y_train_t = torch.FloatTensor(Y_train.values).to(device)
    X_val_t = torch.FloatTensor(X_val).to(device)
    Y_val_t = torch.FloatTensor(Y_val.values).to(device)
    
    train_dataset = TensorDataset(X_train_t, Y_train_t)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    
    # 模型
    model = MTLNetwork(X_train.shape[1]).to(device)
    
    # 類別權重（處理不平衡資料）
    pos_weights = []
    for i in range(Y_train.shape[1]):
        pos_rate = Y_train.iloc[:, i].mean()
        pos_weights.append((1 - pos_rate) / pos_rate)
    pos_weight = torch.FloatTensor(pos_weights).to(device)
    
    criterion = nn.BCELoss(reduction='none')
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    # 訓練
    best_auc = 0
    patience = 10
    patience_counter = 0
    
    for epoch in range(epochs):
        model.train()
        for X_batch, Y_batch in train_loader:
            optimizer.zero_grad()
            outputs = model(X_batch)
            
            # 加權損失
            loss = criterion(outputs, Y_batch)
            weights = torch.where(Y_batch == 1, pos_weight, torch.ones_like(Y_batch))
            loss = (loss * weights).mean()
            
            loss.backward()
            optimizer.step()
        
        # 驗證
        model.eval()
        with torch.no_grad():
            val_outputs = model(X_val_t).cpu().numpy()
            val_aucs = [roc_auc_score(Y_val.iloc[:, i], val_outputs[:, i]) for i in range(3)]
            mean_auc = np.mean(val_aucs)
            
            if mean_auc > best_auc:
                best_auc = mean_auc
                patience_counter = 0
            else:
                patience_counter += 1
                if patience_counter >= patience:
                    break
    
    # 最終預測
    model.eval()
    with torch.no_grad():
        predictions = model(X_val_t).cpu().numpy()
    
    return predictions, model


def train_stl_models(X_train, Y_train, X_val, Y_val, epochs=100, lr=0.001, batch_size=256):
    """訓練 3 個獨立的 STL 模型"""
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    X_train_t = torch.FloatTensor(X_train).to(device)
    X_val_t = torch.FloatTensor(X_val).to(device)
    
    all_predictions = []
    models = []
    
    for task_idx in range(Y_train.shape[1]):
        y_train = Y_train.iloc[:, task_idx].values
        y_val = Y_val.iloc[:, task_idx].values
        
        y_train_t = torch.FloatTensor(y_train).unsqueeze(1).to(device)
        
        train_dataset = TensorDataset(X_train_t, y_train_t)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        
        model = STLNetwork(X_train.shape[1]).to(device)
        
        # 類別權重
        pos_rate = y_train.mean()
        pos_weight = torch.FloatTensor([(1 - pos_rate) / pos_rate]).to(device)
        
        criterion = nn.BCELoss(reduction='none')
        optimizer = optim.Adam(model.parameters(), lr=lr)
        
        best_auc = 0
        patience = 10
        patience_counter = 0
        
        for epoch in range(epochs):
            model.train()
            for X_batch, y_batch in train_loader:
                optimizer.zero_grad()
                outputs = model(X_batch)
                
                loss = criterion(outputs, y_batch)
                weights = torch.where(y_batch == 1, pos_weight, torch.ones_like(y_batch))
                loss = (loss * weights).mean()
                
                loss.backward()
                optimizer.step()
            
            model.eval()
            with torch.no_grad():
                val_pred = model(X_val_t).cpu().numpy().flatten()
                val_auc = roc_auc_score(y_val, val_pred)
                
                if val_auc > best_auc:
                    best_auc = val_auc
                    patience_counter = 0
                else:
                    patience_counter += 1
                    if patience_counter >= patience:
                        break
        
        model.eval()
        with torch.no_grad():
            predictions = model(X_val_t).cpu().numpy().flatten()
        
        all_predictions.append(predictions)
        models.append(model)
    
    return np.column_stack(all_predictions), models

print("訓練函式定義完成")

## 3. 執行 5-Fold 交叉驗證實驗

In [ ]:
# 5-Fold CV（GroupKFold）
n_splits = 5
cv = StratifiedGroupKFold(n_splits=n_splits, shuffle=True, random_state=42)

results = {
    'MTL': {'HTN': [], 'HG': [], 'DL': [], 'time': []},
    'STL': {'HTN': [], 'HG': [], 'DL': [], 'time': []}
}

print("=" * 70)
print("5-Fold CV：MTL vs STL")
print("=" * 70)

for fold, (train_idx, test_idx) in enumerate(cv.split(X, y_htn, groups)):
    print(f"\nFold {fold+1}/{n_splits}")
    
    # 資料分割
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    Y_train, Y_test = Y.iloc[train_idx], Y.iloc[test_idx]
    
    # 驗證無資料洩漏
    train_patients = set(groups.iloc[train_idx])
    test_patients = set(groups.iloc[test_idx])
    assert len(train_patients & test_patients) == 0
    
    # 標準化
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # --- MTL ---
    start_time = time.time()
    mtl_preds, _ = train_mtl_model(X_train_scaled, Y_train, X_test_scaled, Y_test)
    mtl_time = time.time() - start_time
    
    for i, task in enumerate(['HTN', 'HG', 'DL']):
        auc = roc_auc_score(Y_test.iloc[:, i], mtl_preds[:, i])
        results['MTL'][task].append(auc)
    results['MTL']['time'].append(mtl_time)
    
    print(f"  MTL: HTN={results['MTL']['HTN'][-1]:.3f}, HG={results['MTL']['HG'][-1]:.3f}, DL={results['MTL']['DL'][-1]:.3f} ({mtl_time:.1f}s)")
    
    # --- STL ---
    start_time = time.time()
    stl_preds, _ = train_stl_models(X_train_scaled, Y_train, X_test_scaled, Y_test)
    stl_time = time.time() - start_time
    
    for i, task in enumerate(['HTN', 'HG', 'DL']):
        auc = roc_auc_score(Y_test.iloc[:, i], stl_preds[:, i])
        results['STL'][task].append(auc)
    results['STL']['time'].append(stl_time)
    
    print(f"  STL: HTN={results['STL']['HTN'][-1]:.3f}, HG={results['STL']['HG'][-1]:.3f}, DL={results['STL']['DL'][-1]:.3f} ({stl_time:.1f}s)")

print("\n" + "=" * 70)
print("交叉驗證完成")
print("=" * 70)

## 4. 結果摘要

In [ ]:
# 摘要表格
print("=" * 70)
print("結果：MTL vs STL（5-Fold CV）")
print("=" * 70)

summary = []
for method in ['MTL', 'STL']:
    for task in ['HTN', 'HG', 'DL']:
        aucs = results[method][task]
        summary.append({
            '方法': method,
            '任務': task,
            'AUC_mean': np.mean(aucs),
            'AUC_std': np.std(aucs)
        })

summary_df = pd.DataFrame(summary)
print("\nAUC 比較：")
print(summary_df.pivot(index='任務', columns='方法', values='AUC_mean').round(3))

# 訓練時間比較
print(f"\n訓練時間：")
print(f"  MTL：{np.mean(results['MTL']['time']):.1f}s / fold")
print(f"  STL：{np.mean(results['STL']['time']):.1f}s / fold")
print(f"  加速比：{np.mean(results['STL']['time']) / np.mean(results['MTL']['time']):.2f}x")

In [ ]:
# 詳細比較
print("\n" + "=" * 70)
print("詳細比較")
print("=" * 70)

print("\n| 任務 | MTL AUC | STL AUC | 差異 | 勝出 |")
print("|------|---------|---------|------|------|")

for task in ['HTN', 'HG', 'DL']:
    mtl_auc = np.mean(results['MTL'][task])
    stl_auc = np.mean(results['STL'][task])
    diff = mtl_auc - stl_auc
    winner = 'MTL' if diff > 0.005 else 'STL' if diff < -0.005 else '持平'
    print(f"| {task} | {mtl_auc:.3f} | {stl_auc:.3f} | {diff:+.3f} | {winner} |")

# 整體
mtl_avg = np.mean([np.mean(results['MTL'][t]) for t in ['HTN', 'HG', 'DL']])
stl_avg = np.mean([np.mean(results['STL'][t]) for t in ['HTN', 'HG', 'DL']])
print(f"| **平均** | {mtl_avg:.3f} | {stl_avg:.3f} | {mtl_avg-stl_avg:+.3f} | {'MTL' if mtl_avg > stl_avg else 'STL'} |")

In [ ]:
# 儲存結果
results_df = pd.DataFrame([
    {'方法': method, '任務': task, 'Fold': fold+1, 'AUC': auc}
    for method in ['MTL', 'STL']
    for task in ['HTN', 'HG', 'DL']
    for fold, auc in enumerate(results[method][task])
])
results_df.to_csv('../../results/mtl_vs_stl_results.csv', index=False)
print("已儲存：results/mtl_vs_stl_results.csv")

## 5. 結論

### 主要發現

1. **預測效能**：
   - MTL 與 STL 差異約 ~0.3%，無顯著差異

2. **訓練時間**：
   - MTL：較快（共享計算）
   - STL：較慢（3 個獨立模型）

3. **MTL 未能超越 STL 的可能原因**：
   - 三高疾病之間的相關性弱（Phi < 0.1）
   - 樣本數（13,514）對 STL 而言已足夠
   - 各任務難度差異大（HG AUC=0.94 vs HTN AUC=0.74）

### 結論

在本資料集上，STL 與 MTL 表現相近。
當以下情況成立時，MTL 的優勢（共享表徵）有限：
- 任務之間相關性弱
- 資料量足以進行獨立學習